# Numerical Methods Example

## Problem Statement

A classic example, known as the 1D advection equation.
Advection is the transport of some chemical (or any quantity really) by a fluid with a given velocity.

Our goal is to determine the concentration distribution over time.

We introduce the following variables:
- $t:$ time!
- $x:$ this represents the position in space.
In theory, we could consider more coordinates, but this adds complexity.
- $c:$ this is the concentration of the chemical being 'carried' by the fluid.
The concentration varies with space, $x$, and over time, $t$.
In general, this could be any quantity, e.g. mass.
- $\nu:$ the velocity of the fluid.
Can vary with time, $t$, and space, $x$.

To simplify the problem we make the following assumptions:

- The velocity of the fluid is constant (i.e. it doesn't change over time, $t$, and is the same at all positions, $x$).
- Diffusion of the chemical in the fluid is negligible.

Based on the above we can derive the following hyperbolic partial differential equation (PDE):

$$
\frac{\mathrm{d}c(x, t)}{\mathrm{d}t} + \nu\frac{\mathrm{d}c}{\mathrm{d}x}=0
$$


### Initial and boundary conditions